# Cis-eQTL Peak Overlap

**Created**: 14 December 2021

## Environment

In [1]:
rm(list=ls())

In [2]:
library(tidyverse)
library(data.table)
library(GenomicRanges)

setwd("~/eQTL_pQTL_Characterization")

source("03_Functional_Interpretation/scripts/utils/ggplot_theme.R")

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘pack

## Load Data

In [3]:
cis.eqtl <- readRDS("/nfs/users/nfs_n/nm18/gains_team282/eqtl/cisresults/cisqtl_all_significant.rds")

In [4]:
head(cis.eqtl)

snps,gene,statistic,pvalue,beta,se,chr,SNPpos,TSS,threshold
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>
rs4987343,ENSG00000000457,-4.152142,3.720893e-05,-0.04958163,0.01194122,1,169699715,169894267,3.873864e-05
rs4987340,ENSG00000000457,-4.240369,2.546937e-05,-0.05063396,0.01194093,1,169700002,169894267,3.873864e-05
rs2298899,ENSG00000000457,4.176920,3.365090e-05,0.05505924,0.01318178,1,169700591,169894267,3.873864e-05
rs4987328,ENSG00000000457,4.252010,2.432691e-05,0.05597519,0.01316441,1,169701315,169894267,3.873864e-05
rs4987327,ENSG00000000457,-4.218194,2.799604e-05,-0.05023696,0.01190959,1,169701799,169894267,3.873864e-05
rs4987326,ENSG00000000457,-4.218194,2.799604e-05,-0.05023696,0.01190959,1,169701800,169894267,3.873864e-05


In [5]:
geno <- fread("~/gains_team282/Genotyping/All_genotyping_merged_filtered_b38_refiltered_rsID.bim")
colnames(geno) <- c("chr", "snp", "cM", "pos", "minor", "major")

In [6]:
head(geno)

chr,snp,cM,pos,minor,major
<chr>,<chr>,<int>,<int>,<chr>,<chr>
1,rs3131972,0,817341,A,G
1,rs546843995,0,818053,0,G
1,rs553916047,0,818359,0,A
1,1:818740_T_C,0,818740,T,C
1,rs145604921,0,819378,0,C
1,rs535256652,0,821053,0,T


In [7]:
cis.eqtl.conditional <- readRDS("/nfs/users/nfs_n/nm18/gains_team282/eqtl/cisresults/conditionalanalysis/conditional_eQTL_results_final.rds")
cis.eqtl.conditional <- merge(cis.eqtl.conditional, geno, by.x="SNP", by.y="snp") %>%
    dplyr::select(snps=SNP, gene=Gene, pvalue, beta=eQTL_beta, se=eQTL_SE, chr, SNPpos=pos) %>%
    dplyr::mutate(pvalue=as.numeric(pvalue)) %>%
    dplyr::mutate(beta=as.numeric(beta)) %>%
    dplyr::mutate(se=as.numeric(se)) %>%
    dplyr::mutate(chr=as.numeric(chr))

Warning message in gzfile(file, "rb"):
“cannot open compressed file '/nfs/users/nfs_n/nm18/gains_team282/eqtl/cisresults/conditionalanalysis/conditional_eQTL_results_final.rds', probable reason 'No such file or directory'”


ERROR: Error in gzfile(file, "rb"): cannot open the connection


In [ ]:
head(cis.eqtl.conditional)

In [7]:
da.peaks <- read.csv("~/gains_team282/epigenetics/calderon_et_al/analysis/atac_seq/da_peak_set.csv", row.names=1)

In [8]:
head(da.peaks)

,Contrast,Cell.Type,Peak.ID,Peak.Chr,Peak.Start,Peak.End,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,13:50244295-50246227,13,50244295,50246227,4.053709,2.280926,11.056577,7.723960e-21,2.867520e-16,36.11013
2,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,13:79450967-79452794,13,79450967,79452794,3.703510,2.335001,10.812724,3.301762e-20,6.128896e-16,34.73833
3,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,4:38729764-38731699,4,38729764,38731699,2.365172,1.948665,10.304645,6.751549e-19,8.355042e-15,31.88534
4,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,7:140375586-140377343,7,140375586,140377343,3.771741,2.267945,9.507091,7.405967e-17,6.500545e-13,27.43701
5,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,10:7192213-7194489,10,7192213,7194489,2.749342,2.428912,9.478473,8.754943e-17,6.500545e-13,27.27842
6,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,3:71225746-71227945,3,71225746,71227945,2.319326,3.224110,9.424543,1.199727e-16,7.423311e-13,26.97981


In [9]:
de.genes <- read.csv("~/gains_team282/epigenetics/calderon_et_al/analysis/rna_seq/de_genes_set.csv", row.names=1)

In [10]:
head(de.genes)

,Contrast,Cell.Type,Gene.ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000067082,-5.137500,9.255126,-9.215769,8.611657e-16,4.475873e-12,22.99601
2,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000120129,-10.268197,8.818804,-9.202244,9.286043e-16,4.475873e-12,22.93219
3,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000157514,-9.374873,8.953758,-8.388880,8.290944e-14,2.664157e-10,19.11377
4,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000170345,-10.895583,5.741174,-8.301233,1.337386e-13,3.223101e-10,18.70556
5,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000222041,6.292815,5.567182,7.045655,1.056826e-10,2.037560e-07,12.97972
6,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000102962,13.540850,4.331194,6.641714,8.273168e-10,1.205723e-06,11.20495


## Overlap of Sentinel cis-eQTL

First, I'll start by testing the lead SNP for each gene for overlap with DA peaks. These peaks are derived from the initial pass analysis of cis-eQTL (not the conditional analysis). A sentinel SNP is the SNP with the highest p-value for a given location, representing the SNP with the highest evidence for association with the expression of the cognate gene.

In [11]:
da.peak.ranges = GRanges(
    seqnames=da.peaks$Peak.Chr,
    ranges=IRanges(
        start=da.peaks$Peak.Start,
        end=da.peaks$Peak.End
    ),
    strand=rep("*", nrow(da.peaks))
)
mcols(da.peak.ranges) <- da.peaks[,c("Peak.ID", "Cell.Type", "logFC", "adj.P.Val")]

In [12]:
da.peak.ranges

GRanges object with 40825 ranges and 4 metadata columns:
          seqnames              ranges strand |               Peak.ID
             <Rle>           <IRanges>  <Rle> |           <character>
      [1]       13   50244295-50246227      * |  13:50244295-50246227
      [2]       13   79450967-79452794      * |  13:79450967-79452794
      [3]        4   38729764-38731699      * |   4:38729764-38731699
      [4]        7 140375586-140377343      * | 7:140375586-140377343
      [5]       10     7192213-7194489      * |    10:7192213-7194489
      ...      ...                 ...    ... .                   ...
  [40821]       16       159258-161530      * |      16:159258-161530
  [40822]        5   10419748-10421394      * |   5:10419748-10421394
  [40823]        5   49182198-49183584      * |   5:49182198-49183584
  [40824]       17   79415444-79417470      * |  17:79415444-79417470
  [40825]        9 137542676-137544760      * | 9:137542676-137544760
               Cell.Type     logF

In [13]:
cis.eqtl.sentinel <- cis.eqtl %>%
    dplyr::group_by(gene) %>%
    dplyr::slice_min(order_by=pvalue, n=1)

cis.eqtl.sentinel.ranges = GRanges(
    seqnames=cis.eqtl.sentinel$chr,
    ranges=IRanges(
        start=cis.eqtl.sentinel$SNPpos,
        end=cis.eqtl.sentinel$SNPpos
    ),
    strand=rep("*", nrow(cis.eqtl.sentinel))
)
mcols(cis.eqtl.sentinel.ranges) <- cis.eqtl.sentinel[,c("gene", "TSS", "beta", "pvalue")]

In [14]:
cis.eqtl.sentinel.ranges

GRanges object with 17358 ranges and 4 metadata columns:
          seqnames    ranges strand |            gene       TSS       beta
             <Rle> <IRanges>  <Rle> |     <character> <integer>  <numeric>
      [1]        1 169776143      * | ENSG00000000457 169894267  0.0501318
      [2]        1 169776312      * | ENSG00000000457 169894267  0.0501318
      [3]        1 169749095      * | ENSG00000000460 169662007 -0.0719696
      [4]        1 196708768      * | ENSG00000000971 196652043  0.1713546
      [5]        1 196726612      * | ENSG00000000971 196652043  0.1713546
      ...      ...       ...    ... .             ...       ...        ...
  [17354]       11  78283443      * | ENSG00000288538  78223815  -0.246467
  [17355]       11  78284097      * | ENSG00000288538  78223815  -0.246467
  [17356]        1 186609504      * | ENSG00000288562 186624700   0.108611
  [17357]        1 186617709      * | ENSG00000288562 186624700   0.108611
  [17358]        1  43916852      * | ENSG0

In [15]:
overlaps <- findOverlaps(cis.eqtl.sentinel.ranges, da.peak.ranges)

Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': 23
  - in 'y': X, KI270733.1, KI270714.1, GL000220.1, KI270731.1, KI270538.1, KI270465.1, GL000194.1, KI270711.1, Y, KI270728.1, GL000225.1, KI270709.1, KI270744.1, GL000216.2, GL000205.2, GL000195.1, GL000219.1, KI270722.1, GL000009.2, KI270723.1, KI270751.1, KI270750.1, KI270706.1, KI270438.1, KI270736.1, KI270730.1, KI270442.1, KI270737.1, KI270756.1, KI270322.1, KI270727.1
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”


In [16]:
overlap.set <- dplyr::bind_cols(
    as.data.frame(cis.eqtl.sentinel.ranges[overlaps@from, ]) %>%
        dplyr::select(SNP.Chr=seqnames, SNP.Pos=start, eQTL.Gene=gene, eQTL.TSS=TSS, eQTL.Beta=beta, eQTL.P=pvalue),
    as.data.frame(da.peak.ranges[overlaps@to, ]) %>%
        dplyr::select(Peak.ID=Peak.ID, Peak.Chr=seqnames, Peak.Start=start, Peak.End=end, Peak.Width=width, Peak.Cell.Type=Cell.Type, Peak.Log.FC=logFC, Peak.P=adj.P.Val)
)

In [17]:
length(unique(overlap.set$Peak.ID))

[1] 214

In [18]:
length(unique(paste0(overlap.set$SNP.Chr, ":", overlap.set$SNP.Pos)))

[1] 262

In [19]:
length(unique(overlap.set$eQTL.Gene))

[1] 233

## Overlap of All Significant cis-eQTL

I'll test all significant SNPs for each gene for overlap with DA peaks. These peaks are derived from the initial pass analysis of cis-eQTL (not the conditional analysis).

In [20]:
da.peak.ranges = GRanges(
    seqnames=da.peaks$Peak.Chr,
    ranges=IRanges(
        start=da.peaks$Peak.Start,
        end=da.peaks$Peak.End
    ),
    strand=rep("*", nrow(da.peaks))
)
mcols(da.peak.ranges) <- da.peaks[,c("Peak.ID", "Cell.Type", "logFC", "adj.P.Val")]

In [21]:
da.peak.ranges

GRanges object with 40825 ranges and 4 metadata columns:
          seqnames              ranges strand |               Peak.ID
             <Rle>           <IRanges>  <Rle> |           <character>
      [1]       13   50244295-50246227      * |  13:50244295-50246227
      [2]       13   79450967-79452794      * |  13:79450967-79452794
      [3]        4   38729764-38731699      * |   4:38729764-38731699
      [4]        7 140375586-140377343      * | 7:140375586-140377343
      [5]       10     7192213-7194489      * |    10:7192213-7194489
      ...      ...                 ...    ... .                   ...
  [40821]       16       159258-161530      * |      16:159258-161530
  [40822]        5   10419748-10421394      * |   5:10419748-10421394
  [40823]        5   49182198-49183584      * |   5:49182198-49183584
  [40824]       17   79415444-79417470      * |  17:79415444-79417470
  [40825]        9 137542676-137544760      * | 9:137542676-137544760
               Cell.Type     logF

In [22]:
cis.eqtl.ranges = GRanges(
    seqnames=cis.eqtl$chr,
    ranges=IRanges(
        start=cis.eqtl$SNPpos,
        end=cis.eqtl$SNPpos
    ),
    strand=rep("*", nrow(cis.eqtl))
)
mcols(cis.eqtl.ranges) <- cis.eqtl[,c("gene", "TSS", "beta", "pvalue")]

In [23]:
cis.eqtl.ranges

GRanges object with 1549870 ranges and 4 metadata columns:
            seqnames    ranges strand |            gene       TSS       beta
               <Rle> <IRanges>  <Rle> |     <character> <integer>  <numeric>
        [1]        1 169699715      * | ENSG00000000457 169894267 -0.0500981
        [2]        1 169700002      * | ENSG00000000457 169894267 -0.0511467
        [3]        1 169700591      * | ENSG00000000457 169894267  0.0550335
        [4]        1 169701315      * | ENSG00000000457 169894267  0.0559498
        [5]        1 169701799      * | ENSG00000000457 169894267 -0.0508388
        ...      ...       ...    ... .             ...       ...        ...
  [1549866]        9 125750069      * | ENSG00000287782 125426169  0.0281611
  [1549867]        9 125760087      * | ENSG00000287782 125426169  0.0281267
  [1549868]        9 125760246      * | ENSG00000287782 125426169  0.0281267
  [1549869]        9 125824786      * | ENSG00000287782 125426169  0.0277634
  [1549870]      

In [24]:
overlaps <- findOverlaps(cis.eqtl.ranges, da.peak.ranges)

Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': 23
  - in 'y': X, KI270733.1, KI270714.1, GL000220.1, KI270731.1, KI270538.1, KI270465.1, GL000194.1, KI270711.1, Y, KI270728.1, GL000225.1, KI270709.1, KI270744.1, GL000216.2, GL000205.2, GL000195.1, GL000219.1, KI270722.1, GL000009.2, KI270723.1, KI270751.1, KI270750.1, KI270706.1, KI270438.1, KI270736.1, KI270730.1, KI270442.1, KI270737.1, KI270756.1, KI270322.1, KI270727.1
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”


In [25]:
overlap.set <- dplyr::bind_cols(
    as.data.frame(cis.eqtl.ranges[overlaps@from, ]) %>%
        dplyr::select(SNP.Chr=seqnames, SNP.Pos=start, eQTL.Gene=gene, eQTL.TSS=TSS, eQTL.Beta=beta, eQTL.P=pvalue),
    as.data.frame(da.peak.ranges[overlaps@to, ]) %>%
        dplyr::select(Peak.ID=Peak.ID, Peak.Chr=seqnames, Peak.Start=start, Peak.End=end, Peak.Width=width, Peak.Cell.Type=Cell.Type, Peak.Log.FC=logFC, Peak.P=adj.P.Val)
)

There are 3124 unique peaks that have a significant eSNP.

In [26]:
length(unique(overlap.set$Peak.ID))

[1] 3124

There are 9665 eSNPs that overlap with at least one DA peak.

In [27]:
length(unique(paste0(overlap.set$SNP.Chr, ":", overlap.set$SNP.Pos)))

[1] 9665

There are 3958 eGenes that have at least one eSNP present in at least one DA peak.

In [28]:
length(unique(overlap.set$eQTL.Gene))

[1] 3958

In [69]:
da.peak.freqs <- as.data.frame(table(da.peaks$Cell.Type)) %>%
    dplyr::select(Cell.Type=1, DA.Peak.Count=2)

overlap.set.peaks <- overlap.set %>%
    dplyr::select(Peak.ID, Peak.Cell.Type) %>%
    unique()
overlap.freqs <- as.data.frame(table(overlap.set.peaks$Peak.Cell.Type)) %>%
    dplyr::select(Cell.Type=1, Overlap.Count=2) 

merge(da.peak.freqs, overlap.freqs, by="Cell.Type") %>%
    dplyr::mutate(Proportion=Overlap.Count / DA.Peak.Count) %>%
    dplyr::arrange(desc(Proportion))

Cell.Type,DA.Peak.Count,Overlap.Count,Proportion
<fct>,<int>,<int>,<dbl>
Mature_NK,91,32,0.3516484
Effector_CD4pos_T,3465,1157,0.3339105
Monocytes,18,6,0.3333333
Regulatory_T,1682,560,0.3329370
Naive_Teffs,4025,1327,0.3296894
Follicular_T_Helper,2921,941,0.3221499
Effector_memory_CD8pos_T,2410,773,0.3207469
Memory_Tregs,1073,344,0.3205965
Memory_Teffs,4209,1344,0.3193158


## Overlap of Sentinel cis-eQTL (Conditional Analysis)

I can combine the sentinel SNP at each locus with SNPs identified after conditional analysis (conditioning on the sentinel SNP to identify multiple signals).

In [92]:
da.peak.ranges = GRanges(
    seqnames=da.peaks$Peak.Chr,
    ranges=IRanges(
        start=da.peaks$Peak.Start,
        end=da.peaks$Peak.End
    ),
    strand=rep("*", nrow(da.peaks))
)
mcols(da.peak.ranges) <- da.peaks[,c("Peak.ID", "Cell.Type", "logFC", "adj.P.Val")]

In [93]:
da.peak.ranges

GRanges object with 40825 ranges and 4 metadata columns:
          seqnames              ranges strand |               Peak.ID
             <Rle>           <IRanges>  <Rle> |           <character>
      [1]       13   50244295-50246227      * |  13:50244295-50246227
      [2]       13   79450967-79452794      * |  13:79450967-79452794
      [3]        4   38729764-38731699      * |   4:38729764-38731699
      [4]        7 140375586-140377343      * | 7:140375586-140377343
      [5]       10     7192213-7194489      * |    10:7192213-7194489
      ...      ...                 ...    ... .                   ...
  [40821]       16       159258-161530      * |      16:159258-161530
  [40822]        5   10419748-10421394      * |   5:10419748-10421394
  [40823]        5   49182198-49183584      * |   5:49182198-49183584
  [40824]       17   79415444-79417470      * |  17:79415444-79417470
  [40825]        9 137542676-137544760      * | 9:137542676-137544760
               Cell.Type     logF

In [102]:
cis.eqtl.sentinel <- dplyr::bind_rows(
    cis.eqtl %>%
        dplyr::group_by(gene) %>%
        dplyr::slice_min(order_by=pvalue, n=1),
    cis.eqtl.conditional
)

cis.eqtl.sentinel.ranges = GRanges(
    seqnames=cis.eqtl.sentinel$chr,
    ranges=IRanges(
        start=cis.eqtl.sentinel$SNPpos,
        end=cis.eqtl.sentinel$SNPpos
    ),
    strand=rep("*", nrow(cis.eqtl.sentinel))
)
mcols(cis.eqtl.sentinel.ranges) <- cis.eqtl.sentinel[,c("gene", "beta", "pvalue")]

In [104]:
cis.eqtl.sentinel.ranges

GRanges object with 33746 ranges and 3 metadata columns:
          seqnames    ranges strand |            gene       beta      pvalue
             <Rle> <IRanges>  <Rle> |     <character>  <numeric>   <numeric>
      [1]        1 169776143      * | ENSG00000000457  0.0501318 4.86024e-12
      [2]        1 169776312      * | ENSG00000000457  0.0501318 4.86024e-12
      [3]        1 169749095      * | ENSG00000000460 -0.0719696 6.26471e-09
      [4]        1 196708768      * | ENSG00000000971  0.1713546 7.63523e-06
      [5]        1 196726612      * | ENSG00000000971  0.1713546 7.63523e-06
      ...      ...       ...    ... .             ...        ...         ...
  [33742]        4  81171813      * | ENSG00000138669 -0.0764252 1.30521e-10
  [33743]        4  76196421      * | ENSG00000138760  0.0763221 5.05314e-08
  [33744]        4 163514546      * | ENSG00000273449  0.0848184 2.26190e-23
  [33745]        4    121772      * | ENSG00000275426 -0.0907441 7.39485e-18
  [33746]        1 

In [105]:
overlaps <- findOverlaps(cis.eqtl.sentinel.ranges, da.peak.ranges)

Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': 23
  - in 'y': X, KI270733.1, KI270714.1, GL000220.1, KI270731.1, KI270538.1, KI270465.1, GL000194.1, KI270711.1, Y, KI270728.1, GL000225.1, KI270709.1, KI270744.1, GL000216.2, GL000205.2, GL000195.1, GL000219.1, KI270722.1, GL000009.2, KI270723.1, KI270751.1, KI270750.1, KI270706.1, KI270438.1, KI270736.1, KI270730.1, KI270442.1, KI270737.1, KI270756.1, KI270322.1, KI270727.1
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”


In [107]:
overlap.set <- dplyr::bind_cols(
    as.data.frame(cis.eqtl.sentinel.ranges[overlaps@from, ]) %>%
        dplyr::select(SNP.Chr=seqnames, SNP.Pos=start, eQTL.Gene=gene, eQTL.Beta=beta, eQTL.P=pvalue),
    as.data.frame(da.peak.ranges[overlaps@to, ]) %>%
        dplyr::select(Peak.ID=Peak.ID, Peak.Chr=seqnames, Peak.Start=start, Peak.End=end, Peak.Width=width, Peak.Cell.Type=Cell.Type, Peak.Log.FC=logFC, Peak.P=adj.P.Val)
)

There are 309 unique peaks that have a sentinel eSNP.

In [108]:
length(unique(overlap.set$Peak.ID))

[1] 309

There are 375 sentinel eSNPs that overlap with at least one DA peak.

In [109]:
length(unique(paste0(overlap.set$SNP.Chr, ":", overlap.set$SNP.Pos)))

[1] 375

There are 348 eGenes that have at least one eSNP present in at least one DA peak.

In [110]:
length(unique(overlap.set$eQTL.Gene))

[1] 348